**Otras formas de analizar el supuesto de la clase pasada (linealidad y cambio estructural)**

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms

KeyboardInterrupt: 

In [ ]:
data=pd.read_csv("________________", delimiter="\t")

In [ ]:
Modelo1=sm.OLS(data["CM"],sm.add_constant(data[["PGNP","FLR"]]))
Resultados1=Modelo1.fit()
print(Resultados1.summary())

**Supuesto de Cambio Estructural**

In [ ]:
Valores_estimados=Resultados1.predict()
Residuos=Resultados1.resid

In [ ]:
CambioEstructural = sms.______________(Residuos) # Prueba de cambio estructural con residuos de MCO
NombresCE = ["Estadístico", "P-valor", "Valores Críticos"]

In [ ]:
pd.DataFrame({"CUSUM" : __________}, index=__________) # Ho = No hay Cambio estructural 

## Análisis sobre los errores del modelo

In [ ]:
fig,ax=plt.subplots()
ax.scatter(_______________,____________,color="blue")
ax.set_xlabel("Valores estimados")
ax.set_ylabel("Residuos")
ax.set_title("Residuos vs Valores estimados",fontsize=20)
ax.axhline(color = "red")
plt.show()

**Supuesto de Linealidad**

In [ ]:
# H0 = Linealidad

ResultadosTest = sms.____________(Resultados1)
Nombres = ["Estadistico","p-valor"]
pd.Series(_________, index =_____________)

In [ ]:
pd.DataFrame(pd.Series(____________, index = ___________), columns = ["Harvey-Collier"])

In [ ]:
ResultadosRainbow= sms.___________(____________) 

In [ ]:
Nombres2 = ["Estadístico","p-valor"]

In [ ]:
pd.DataFrame({"Rainbow":_______________}, index = __________) # Ho = linealidad

### Media de los residuos

In [ ]:
np.__________

**Supuesto 4: Homoscedasticidad**

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Residuos")
ax.plot(__________)
ax.set_xlabel("Observaciones")
ax.set_ylabel("Observaciones")
plt.show()

In [ ]:
Explicativas = Resultados1.___________
Test = sms._______________(_______, __________)
NombresTest = ["Estadístico LM", "P-value LM", "Estadístico F", "P-value F"] 
pd.Series(Test, index = NombresTest) 

# Ho = homoscedasticidad

In [ ]:
pd.DataFrame({"BreushPagan":__________}, index = ________)

In [ ]:
pd.DataFrame(pd.Series(________, index = ____________), columns = ["BreushPagan"])

In [ ]:
TestWhite = sms._____________(__________,_________)
NombresTest = ["Estadístico LM", "P-value LM", "Estadístico F", "P-value F"]
pd.Series(________, index = ________)

**Supuesto de No Autocorrelación**

In [ ]:
ResultadosDW = sms.____________
NombresDW= ["Estadístico DW"]
pd.Series(__________, _____________)

In [ ]:
ResultadoBreush = sms.______________________(__________, nlags = __)
pd.Series(__________, index = __________)

In [ ]:
ResultadosJungBox= sms.____________(__________, lags = 1, 
                                       boxpierce = ________)
NombresJB = ["Estadístico LB", "p-value LB", "Estadístico BP", "p-value BP"]
pd.Series(_________, ___________)

**Supuesto de normalidad**

In [ ]:
# Las Ho = no normalidad

In [ ]:
ResultadosTestJB= sms.___________(_________)
Nombres = ["JB", "p-value","Asimetria","Curtosis"]
pd.Series(______________, index = Nombres)

In [ ]:
ResultadosTestAD = sms._________(_________)
Nombres=["AD","p-valoe AD"]
pd.Series(___________, index = _______)

In [ ]:
ResultadosTestKS= sms.______________(__________)
Nombres = ["KS", "p-valor KS"]
pd.Series(____________, index = __________)

In [ ]:
fig, ax = plt.subplots()
probplot = sm.__________(__________)
probplot.qqplot(_______, _______)
ax.set_title("QQ Plot", color="darkblue")
print()